In [101]:
import os
import pandas as pd
import string

In [6]:
# set the path for reading the Compustat lists
# path = 'D:\\studyproject\\bankruptcy\\data\\compustat\\' # for win decomment this line
path = '/Users/user/Documents/Bankruptcy/bankruptcy/data/Eikon/ \\
        Identifiers_Mapping/0.Ticker_CUSIP-to-ISIN_RIC/' # for mac decomment this line

# read the lists from the previous conversion
columns = ['Company', 'CUSIP', 'ISIN', 'ISINc', 'RIC', 'RICc']
bankrupt = pd.read_csv(path + '0.bankrupt_list.csv', usecols=columns, dtype=object)
healthy = pd.read_csv(path + '0.healthy_list.csv', usecols=columns, dtype=object)

print(bankrupt.head())
healthy.head()

                        Company      CUSIP          ISIN ISINc           RIC  \
0  Amber Resources Company of C  023184203  US0231842032     1           NaN   
1         Fairchild Corp. (The)  303698104  US3036981047     0  FCHDQ.PK^K11   
2     Constar International Inc  21036U206  US21036U2069     1           NaN   
3  Energy Conversion Devices In  292659109  US2926591098     0  ENERQ.PK^I12   
4    Fleetwood Enterprises Inc.  339099103  US3390991038     0  FLTWQ.PK^H10   

  RICc  
0    0  
1    1  
2    0  
3    1  
4    1  


,Company,CUSIP,ISIN,ISINc,RIC,RICc
0,AAR Corp,000361105,US0003611052,0,AIR,0
1,ADC Telecommunications Inc.,000886309,US0008863096,1,NaN,0
2,AFP Imaging Corp,001058205,US0010582056,0,IWKS.PK,1
3,Alpharma Inc.,020813101,US0208131013,1,NaN,0
4,American Airlines Group Inc,02376R102,US02376R1023,0,AAL.O - AAL.Z,3


In [98]:
# implement the CUSIP to ISIN formula

def c2i(CUSIP):
    '''
    This function takes CUSIP and returns
    the ISIN codes, for the cases the company
    is US- or Canada-based.
    '''
    usISIN = '3028'+ CUSIP
    caISIN = '1210' + CUSIP
    
    def final_digit(ISIN2B):
        '''
        This function takes 'country code+CUSIP' (ISIN-to-be),
        and calculate the final digit of corresponding
        ISIN code.
        '''
        ISIN = []
        for ch in ISIN2B:
            try:
                ISIN.append(int(ch))
            except:
                ISIN.append((ord(ch) - 55) // 10)
                ISIN.append((ord(ch) - 55) % 10)
        
        
        counter = -1
        list_A = []
        list_B = []
        while counter >= -len(ISIN):
            list_A.append(ISIN[counter])
            try:        
                list_B.append(ISIN[counter-1])
            except:
                pass
            counter -= 2
        
        list_A = [2*x for x in list_A]
        
        for i in list_A:
            if i > 9:
                list_A.append(i // 10)
                list_A.append(i % 10)
                list_A.remove(i)
        
        final_digit = (10 - (sum(list_A) + sum(list_B)) % 10) % 10
        
        return final_digit
         
    # if the company is US-based
    usISIN = 'US' + CUSIP + str(final_digit(usISIN))
    # and if it is Canada-based
    caISIN = 'CA' + CUSIP + str(final_digit(caISIN))
    
    return usISIN, caISIN
    

In [ ]:
# convert lists CUSIPs to ISINs
def createISIN(df):
    '''
    this functions takes bankrupt and healthy dataframes,
    use the CUSIP column and creates corresponding ISINs,
    and compare the consistency with the existing ISINS
    '''
    # add an empty column for CUSIP-converted ISIN
    df['cusipISIN'] = ''

    # iterate over dataframe rows
    for index, row in df.iterrows():
        # convert CUSIP to ISIN
        usISIN, caISIN = c2i(row['CUSIP'])
        
        # 4 different cases is possible:
        
        # case1-2. no inconsistency
        if row['ISINc'] == 0:
            # case1. no ISIN
            if row['ISIN'] != row['ISIN']:
                row['ISIN'] = 


                